In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

CONNECT_PANDA = True
CONNECT_MOBILE = False

IP_CUR = "192.168.17.4"# get_ip_address()
MOBILE_IP = "192.168.0.102"
PANDA_HOST_IP = "192.168.17.2"
PANDA_ROBOT_IP = "192.168.17.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

from concurrent import futures
import logging
import math
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt

import grpc
import RemoteCam_pb2
import RemoteCam_pb2_grpc

MAX_MESSAGE_LENGTH = 10000000
PORT_CAM = 10509

Current PC IP: 192.168.17.4
Mobile ROB IP: 192.168.0.102


In [5]:
PANDA_BASE_OFFSET = (0.172,0,0.439)
PANDA_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6
COL_COLOR = (1,1,1,0.2)
    
# if EXP_SCENARIO == ExpType.REMOVE_OBS: ## Obstacle removing
#     BAG_COUNT = 5
#     CLOCK_COUNT = 0
#     TARGET_COUNT = 5
#     LOG_FORCE = False
# else: ## Contact 
#     BAG_COUNT = 3
#     CLOCK_COUNT = 3
#     TARGET_COUNT = 5
#     LOG_FORCE = True

from pkg.controller.combined_robot import *
from pkg.project_config import *


if not CONNECT_PANDA:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.panda, 
                           (PANDA_BASE_OFFSET, PANDA_BASE_RPY),
                           "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP), root_on="kmb0_platform")

ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[CONNECT_MOBILE, CONNECT_PANDA])

connection command:
kmb0: False
panda1: True


In [6]:
from util import *
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene
from pkg.geometry.geometry import GeometryItem
from pkg.geometry.geotype import GEOTYPE

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)

gscene.set_workspace_boundary(
    -1, 4, -2.5, 2.5, -CLEARANCE, WS_HEIGHT, thickness=WALL_THICKNESS)

viewpoint = add_cam(gscene, tool_link="panda1_link8")
gscene.show_pose(crob.get_real_robot_pose())

ros_node already initialized somewhere else
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### MultiICP detector

In [9]:
from pkg.detector.multiICP.multiICP import MultiICP, MultiICP_Obj
from pkg.detector.multiICP.config import *
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

import open3d as o3d
import numpy as np

CONNECT_CAM = True

In [10]:
realsense = RealSense()

# from demo_utils.data_reconstructed_camera import DataRecontructedCamera
# dcam = DataRecontructedCamera(crob, viewpoint)
# if not CONNECT_CAM:
#     dcam.initialize()
    
if CONNECT_CAM:
    micp = MultiICP(realsense)
    micp.initialize()
#     dcam.ready_saving(*realsense.get_config())
#     cam_pose = viewpoint.get_tf(VIEW_POSE_EXT)
else:
    # use manually given camera configs
    config_list, img_dim = load_pickle(RNB_PLANNING_DIR+"release/multiICP_data/cam_configs.pkl")
    micp.initialize(config_list, img_dim)
    micp = MultiICP(dcam)
    micp.initialize()

Start streaming
Initialize Done


### Shared Detector

In [11]:
from pkg.utils.shared_function import clear_channels_on, sa
clear_channels_on("SharedDetector")

from pkg.detector.multiICP.shared_detector import SharedDetectorGen
sd = SharedDetectorGen(tuple(reversed(micp.dsize))+(3,))()
sd.init()

### set ICP config

In [12]:
# Load config file of object information
obj_info_dict = get_obj_info()

In [13]:
# initial guess는 실험하는거 보면서 수정?
micp_suitcase = MultiICP_Obj(obj_info_dict["suitcase"], None,
                        OffsetOnModelCoord("suitcase", R=Rot_axis(1, np.pi/2), offset=(0.,0.,0.)))


micp_clock = MultiICP_Obj(obj_info_dict["clock"], None,
                        OffsetOnModelCoord("clock", R=Rot_axis(1, np.pi/2), offset=(-0.07,0.03,0.)))


micp_table = MultiICP_Obj(obj_info_dict["dining table"], None,
                        OffsetOnModelCoord("dining table", R=np.identity(3), offset=(0.,0.,0.)))

In [14]:
micp_dict = {"suitcase": micp_suitcase, "clock": micp_clock, "dining table": micp_table}
# micp_dict = {"clock": micp_clock}
# micp_dict = {"dining table": micp_table}
# micp_dict = {"suitcase": micp_suitcase}

In [15]:
# set config information for micp
micp.set_config(micp_dict, sd, crob, viewpoint)

### Detect

In [16]:
micp.set_ICP_thres(thres_ICP=0.1, thres_front_ICP=0.05)
pose_dict = micp.detect(name_mask=["clock"], visualize=True)

name_mask is ['clock']
===== Detected : clock, 2 object(s) =====

'clock' is not in gscene. Use manual input for initial guess

Apply point-to-point ICP
Total ICP Transformation is:
[[ 9.93990362e-01  7.49974591e-02 -7.97404647e-02 -6.36842319e-02]
 [-7.95705473e-02 -5.25570750e-03 -9.96815382e-01  2.23177969e-01]
 [-7.51777134e-02  9.97169875e-01  7.43466261e-04  7.54723971e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
initial: 
[[ 0.99  0.07 -0.08 -0.06]
 [-0.08 -0.01 -1.    0.22]
 [-0.08  1.    0.    0.75]
 [ 0.    0.    0.    1.  ]]
Apply point-to-point ICP
Total ICP Transformation is:
[[ 0.9880846   0.07967176 -0.13168611 -0.06045181]
 [-0.13631939  0.05579589 -0.98909243  0.23298662]
 [-0.07145519  0.99525837  0.06599187  0.74940088]
 [ 0.          0.          0.          1.        ]]
result: 
[[ 0.99  0.08 -0.13 -0.06]
 [-0.14  0.06 -0.99  0.23]
 [-0.07  1.    0.07  0.75]
 [ 0.    0.    0.    1.  ]]
Found 6DoF pose of clock_1

'clock' is not in gscene.

In [ ]:
pose_dict = micp.detect(name_mask=["suitcase", "clock", "dining table"])

### Update Object to scene

In [ ]:
# add table
center, rpy = pose_refine("dining table", pose_dict["dining table"])
table_vis = add_table(gscene, "table", center, rpy)

In [ ]:
# add or update suitcase
add_update_object(gscene, crob, "suitcase", pose_dict, height = 0)

In [17]:
# add or update clock
add_update_object(gscene, crob, "clock", pose_dict, separate_dist=0.2, height = 0.734)

Total 0 clock in the scene
Add new clock in the scene
Add new clock in the scene


### Test code for multiple instance detection
* MultiICP Detector 이전까지 실행시키고 아래의 코드 실행
* Separte distance는 수동으로 조절

In [5]:
# Origianlly, 4 suitcases are existed in scene
c10 = add_carrier(gscene, "suitcase_0", (1.,1.,0), (0,0,0))
c11 = add_carrier(gscene, "suitcase_1", (1.,2.,0), (0,0,0))
c12 = add_carrier(gscene, "suitcase_2", (2.,1.,0), (0,0,0))
c13 = add_carrier(gscene, "suitcase_3", (2.,2.,0), (0,0,0))

In [6]:
# Detection result, but T1, T2 are already detected objects so, these are in the scene
# T3 is newly detected object so this is not in the scene
# Therefore, 1 and 2 should be updated, but 3 should be added

T1 = SE3(Rot_axis(3, np.pi/10), (1.1,1.18,0.3))
T2 = SE3(np.identity(3), (2.1,1.88,-0.1))
T3 = SE3(Rot_axis(1, -np.pi/5) , (3.1,1.9,-0.2))
pose_dict = {"suitcase_0":T1, "suitcase_1":T2, "suitcase_2":T3}

In [7]:
add_update_object(gscene, crob, "suitcase", pose_dict)

Total 4 suitcase in the scene
Update existing suitcase in the scene
Update existing suitcase in the scene
Add new suitcase in the scene
